In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame()
df = pd.read_csv("train_proper_csv_attack_types.csv",header = None)
df = df.rename(columns=df.iloc[0])
df = df.drop(df.index[0])
df = df.drop('unknown2',axis=1)

C:\Users\Rupesh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df1 = pd.DataFrame()
df1 = pd.read_csv("test_proper_csv_attack_types.csv",header = None)
df1 = df1.rename(columns=df1.iloc[0])
df1 = df1.drop(df1.index[0])
df1 = df1.drop('unknown2',axis=1)

C:\Users\Rupesh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_tot = pd.concat([df,df1])

In [5]:
def bool_attack(x):
    if(x != "normal"):
        return "attack"
    else:
        return "normal"

In [6]:
from sklearn.feature_selection import SelectFpr
from sklearn.feature_selection import chi2
from sklearn.preprocessing import Normalizer

df_tot_1 = df_tot.drop('unknown1',axis=1)
df_output = df_tot['unknown1'].apply(bool_attack)
df_tot_dummies = pd.get_dummies(df_tot_1,columns = ["protocol_type","service","flag"])
x_normalise = Normalizer().fit(df_tot_dummies)
df_tot_dummies = x_normalise.transform(df_tot_dummies)
x_new = SelectFpr(chi2, alpha=0.05).fit_transform(df_tot_dummies, df_output)

In [7]:
import pickle

random_forest_clf = pickle.load(open('random_forest_test.sav', 'rb'))
AdaBoost_clf = pickle.load(open('AdaBoost_test.sav', 'rb'))
svc_clf = pickle.load(open('svc_test.sav', 'rb'))

In [8]:
X_train = x_new[:125000]
y_train = df_output.iloc[:125000]
X_test = x_new[125000:]
y_test = df_output.iloc[125000:]

In [9]:
from sklearn.metrics import classification_report

In [10]:
def convert_bool(x):
    if(x == "attack"):
        return 1
    else:
        return 0
    
y_train = y_train.apply(convert_bool)

In [29]:
type(y_train)

pandas.core.series.Series

In [11]:
rf_pred = random_forest_clf.predict(X_train)
rf_conf = random_forest_clf.predict_proba(X_train)
ada_pred = AdaBoost_clf.predict(X_train)
ada_conf = AdaBoost_clf.predict_proba(X_train)
svc_pred = svc_clf.predict(X_train)
svc_conf = svc_clf.predict_proba(X_train)

In [12]:
conf_vec_list=[]
clf_vec_list=[]
state_vec_list=[]

In [13]:
for i in range(len(rf_pred)):
    tmp1=[]
    tmp1.append(convert_bool(rf_pred[i]))
    tmp1.append(convert_bool(ada_pred[i]))
    tmp1.append(convert_bool(svc_pred[i]))
    clf_vec_list.append(tmp1)
    state_vec_list.append(tmp1+list(x_new[i]))
    
    tmp1=[]
    tmp1.append(rf_conf[i])
    tmp1.append(ada_conf[i])
    tmp1.append(svc_conf[i])
    conf_vec_list.append(tmp1)

In [14]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [48]:
len(x_new[0])

36

In [15]:
# create model
model = Sequential()
model.add(Dense(100, input_dim=39, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

In [21]:
model.predict(np.array(state_vec_list[0:10]))

array([[ 0.52624005,  0.94977194,  0.9498325 ],
       [ 0.52054763,  0.94958097,  0.9500668 ],
       [ 0.56507987,  0.94988036,  0.94995481],
       [ 0.52292168,  0.94981515,  0.94980353],
       [ 0.53459072,  0.94930536,  0.94936877],
       [ 0.56540352,  0.94961786,  0.94976634],
       [ 0.56308681,  0.95012391,  0.95018131],
       [ 0.56480825,  0.9499734 ,  0.95006561],
       [ 0.56377184,  0.95024449,  0.95023865],
       [ 0.56386727,  0.95014137,  0.95019698]], dtype=float32)

In [16]:
# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [17]:
input_list = []
output_list = []

In [18]:
def get_rew(qval_v,y,conf_v,clf_v):
    
    act_qval=[]
    
    for j in range(3):
        
        if(y == clf_v[j]):
            #print(qval_v[j])
            #print(conf_v[j]/5)
            tmp = qval_v[j] + conf_v[j][y]/5
            
            if(tmp>1):
                tmp = 0.95

            act_qval.append(tmp)
        else:
            tmp = qval_v[j] - conf_v[j][abs(1-y)]/5
            if(tmp<0):
                tmp = 0.05

            act_qval.append(tmp)
    
    return act_qval

In [19]:
def get_batch(ind_list, input_list, output_list):
    x=[]
    y=[]
    for item in ind_list:
        x.append(input_list[item])
        y.append(output_list[item])
    
    return x,y

In [ ]:
import random

for i in range((int)(len(state_vec_list)/10)):
    
    print("itteration: " + str(i))
    input_list = input_list + state_vec_list[i*10:(i+1)*10]
    Q_val_list = model.predict(np.array(state_vec_list[i*10:(i+1)*10]))
    act_out_list = []
    
    for j in range(len(Q_val_list)):
        #print(conf_vec_list[i*10+j])
        act_out_list.append(get_rew(Q_val_list[j],y_train[i*10+j+1],conf_vec_list[i*10+j],clf_vec_list[i*10+j]))
        
    model.train_on_batch(np.array(state_vec_list[i*10:(i+1)*10]), np.array(act_out_list))
    
    output_list = output_list + act_out_list
    #print(len(input_list))
    #print(len(output_list))
    for j in range(10):
        ind_list = random.sample(range(0, len(input_list)), 10)
        x, y = get_batch(ind_list, input_list, output_list)
        
        model.train_on_batch(np.array(x), np.array(y))

In [ ]:
# Fit the model
model.fit(np.array(state_vec_list), np.array(y_train), epochs=30, batch_size=10)

In [23]:
rf_pred_ts = random_forest_clf.predict(X_test)
ada_pred_ts = AdaBoost_clf.predict(X_test)
svc_pred_ts = svc_clf.predict(X_test)

In [24]:
input_vec = []

In [25]:
for i in range(len(rf_pred_ts)):
    tmp1=[]
    tmp1.append(convert_bool(rf_pred_ts[i]))
    tmp1.append(convert_bool(ada_pred_ts[i]))
    tmp1.append(convert_bool(svc_pred_ts[i]))
    input_vec.append(tmp1+list(X_test[i]))
    
y_test = y_test.apply(convert_bool)

In [38]:
ada_pred_ts

array(['attack', 'normal', 'attack', ..., 'attack', 'normal', 'attack'], dtype=object)

In [26]:
y_pred = model.predict(np.array(input_vec))

In [39]:
print(rf_pred_ts[0])
print(ada_pred_ts[0])
print(svc_pred_ts[0])
print(y_pred[0])

attack
attack
attack
[ 0.56398749  0.95026147  0.95035213]


In [65]:
y_pred_1=[]

In [ ]:
for k in range(len(y_pred)):
    ac=0
    nc=0
    if(y_pred[k][0] > 0.5):
        if(convert_bool(rf_pred_ts[k]) == 1):
            ac = ac+1
        else:
            nc = nc+1
    
    if(y_pred[k][1] > 0.8):
        if(convert_bool(ada_pred_ts[k]) == 1):
            ac = ac+1
        else:
            nc = nc+1
    
    if(y_pred[k][2] > 0.8):
        if(convert_bool(svc_pred_ts[k]) == 1):
            ac = ac+1
        else:
            nc = nc+1
    
    print(ac-nc)
    if(ac > nc):
        print("attack")
        y_pred_1.append(1)
    else:
        print("norm")
        y_pred_1.append(0)

In [63]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [69]:
def cal_fpr_acc(TP,FP,TN,FN):
    acc = (TP+TN)/(TP+TN+FP+FN)
    fpr = (FP)/(FP+TN)
    
    return fpr, acc

In [71]:
params = list(perf_measure(np.array(y_test),y_pred_1))
cal_fpr_acc(params[0],params[1],params[2],params[3])

(0.026164209704188226, 0.8107671372682429)

In [40]:
# evaluate the model
scores = model.evaluate(np.array(input_vec), y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

23264/23516 [============================>.] - ETA: 0s
acc: 81.07%


In [41]:
from sklearn.metrics import confusion_matrix

In [47]:
y_pred = model.predict(np.array(input_vec))



In [49]:
perf_measure(np.array(y_test),y_pred)

(7151, 232, 9348, 3642)

In [22]:
model.predict(np.array(state_vec_list[30:50]))

array([[ 0.58925104,  0.94940388,  0.94942933],
       [ 0.56969208,  0.94931269,  0.94956756],
       [ 0.52008027,  0.94968128,  0.94975591],
       [ 0.56361681,  0.95003319,  0.95029205],
       [ 0.54490072,  0.94995046,  0.94972837],
       [ 0.51979095,  0.94827014,  0.94873452],
       [ 0.54613537,  0.95046735,  0.95027506],
       [ 0.56381893,  0.95011669,  0.95015258],
       [ 0.5239948 ,  0.94944054,  0.94952422],
       [ 0.56291771,  0.95010185,  0.95014113],
       [ 0.56979054,  0.94930077,  0.94955927],
       [ 0.56214434,  0.9501701 ,  0.95020354],
       [ 0.52084476,  0.94964552,  0.94996887],
       [ 0.50787294,  0.94820678,  0.94853979],
       [ 0.52398324,  0.94946116,  0.94959015],
       [ 0.56421059,  0.95007795,  0.95017695],
       [ 0.5739041 ,  0.95045245,  0.95057786],
       [ 0.52979797,  0.94924563,  0.9493807 ],
       [ 0.57302642,  0.11274257,  0.12911497],
       [ 0.56214583,  0.95018411,  0.95022792]], dtype=float32)